### Replicate Figure 5 from Deep Double Descent

Tasks
*  Train 3 Layer CNNs of widths 4, 8, 12, 16, ..., 32, 46, ..., 128, 192, 256 with 500K Steps of SGD on CIFAR-10 without data-augmentation, and with 10% added label noise.
*  Train 5 Layer CNNs of widths 4, 8, 12, 16, ..., 32, 46, 64 with 500K Steps of SGD on CIFAR-10 without data-augmentation, and with 10% added label noise.
*  Train 3 Layer CNNs of widths 2, 4, 6, ..., 14, 16 with 500K Steps of SGD on CIFAR-10 without data-augmentation, and with 10% added label noise.

Future Tasks:
*  Redo parts of the above using the lazy and active regime.

Notes:
* Depth 3 conv net with width 256 has approx 1.2 million parameters. 
* Depth 5 conv net with width 64 has approx 1.5 million parameters.
* depth 7 conv net with width 16 has approx 1.5 million parameters.

Interpolation is reached at ~600,000 parameters.
We want to cross compare # parameters, not the width value.


### This note book contains results from training depth 3 networks.

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

from models.conv_nets import make_convNet 
from utils.train_utils import timer, inverse_squareroot_lr

# keeps tensorflow from using all available GPU memory when a model is initialized.
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

#### Train depth 5 conv nets at widths of [1, 2,3, ..., 64]

In [ ]:
batch_size=128
n_epochs = 500_000 // (50_000 // 128) # total number desirec SGD steps / number batches per epoch
label_noise = 0.10

cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# apply label noise to the data set
random_idx = np.random.choice(x_train.shape[0], int(label_noise*x_train.shape[0]))
rand_labels = np.random.randint(low=y_train.min(), high=y_train.max(), size=len(random_idx))
y_train[random_idx] = np.expand_dims(rand_labels, axis=1)

image_shape = x_train[0].shape

model_histories = {}
convnet_widths = [i + 1 for i in range(64)]
convnet_depth = 5

# Paths to save model weights and 
model_weights_paths = f'trained_model_weights/conv_nets_depth_{convnet_depth}/'
data_save_path = 'experimental_results/conv_nets_'

for width in convnet_widths:
    # Depth K Conv Net using default Kaiming Uniform Initialization.
    conv_net, model_id = make_convNet(image_shape, depth=convnet_depth, init_channels=width)

    conv_net.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=inverse_squareroot_lr()),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model_timer = timer()
    
    print(f'STARTING TRAINING: {model_id}')
    history = conv_net.fit(
        x=x_train, y=y_train, 
        validation_data=(x_test, y_test),
        epochs=n_epochs,
        batch_size=batch_size,
        verbose=0, 
        callbacks = [model_timer]
    )
    print(f'FINISHED TRAINING: {model_id}')
    
    # add results to dictionary and store the resulting model weights.
    model_histories[model_id] = history
    history.model.save_weights(model_weights_paths+model_id)
    
    # clear GPU of prior model to decrease training times.
    tf.keras.backend.clear_session()
    
    
# store all the results in a pickle file for later graphing and analysis.
depth_5_metrics = {} 

for m_id, history in model_histories.items():
    depth_5_metrics[m_id] = history.history
    
pkl.dump(depth_5_metrics, open(data_save_path + f'depth_{convnet_depth}.pkl', 'wb'))

STARTING TRAINING: conv_net_depth_5_width_1
Epoch: 0000, Total Run Time: 00:00:02 - Loss: 2.1436e+00, Accuracy: 0.205, Test Loss: 2.1708e+00, Test Accuracy: 0.189
Epoch: 0025, Total Run Time: 00:00:30 - Loss: 1.7699e+00, Accuracy: 0.374, Test Loss: 1.6654e+00, Test Accuracy: 0.391
Epoch: 0050, Total Run Time: 00:00:58 - Loss: 1.7345e+00, Accuracy: 0.389, Test Loss: 1.6063e+00, Test Accuracy: 0.419
Epoch: 0075, Total Run Time: 00:01:26 - Loss: 1.7173e+00, Accuracy: 0.396, Test Loss: 1.6434e+00, Test Accuracy: 0.394
Epoch: 0100, Total Run Time: 00:01:55 - Loss: 1.7063e+00, Accuracy: 0.404, Test Loss: 1.6059e+00, Test Accuracy: 0.409
Epoch: 0125, Total Run Time: 00:02:23 - Loss: 1.7000e+00, Accuracy: 0.407, Test Loss: 1.6874e+00, Test Accuracy: 0.391
Epoch: 0150, Total Run Time: 00:02:51 - Loss: 1.6956e+00, Accuracy: 0.410, Test Loss: 1.5710e+00, Test Accuracy: 0.430
Epoch: 0175, Total Run Time: 00:03:20 - Loss: 1.6888e+00, Accuracy: 0.413, Test Loss: 1.5822e+00, Test Accuracy: 0.421
Epoc

Epoch: 0400, Total Run Time: 00:07:34 - Loss: 1.4015e+00, Accuracy: 0.541, Test Loss: 1.2752e+00, Test Accuracy: 0.550
Epoch: 0425, Total Run Time: 00:08:02 - Loss: 1.3994e+00, Accuracy: 0.541, Test Loss: 1.2585e+00, Test Accuracy: 0.563
Epoch: 0450, Total Run Time: 00:08:31 - Loss: 1.3995e+00, Accuracy: 0.542, Test Loss: 1.2784e+00, Test Accuracy: 0.549
Epoch: 0475, Total Run Time: 00:08:59 - Loss: 1.4001e+00, Accuracy: 0.540, Test Loss: 1.2988e+00, Test Accuracy: 0.540
Epoch: 0500, Total Run Time: 00:09:28 - Loss: 1.3987e+00, Accuracy: 0.542, Test Loss: 1.2577e+00, Test Accuracy: 0.558
Epoch: 0525, Total Run Time: 00:09:56 - Loss: 1.3980e+00, Accuracy: 0.542, Test Loss: 1.2580e+00, Test Accuracy: 0.556
Epoch: 0550, Total Run Time: 00:10:25 - Loss: 1.3971e+00, Accuracy: 0.542, Test Loss: 1.2733e+00, Test Accuracy: 0.553
Epoch: 0575, Total Run Time: 00:10:53 - Loss: 1.3968e+00, Accuracy: 0.541, Test Loss: 1.2656e+00, Test Accuracy: 0.557
Epoch: 0600, Total Run Time: 00:11:21 - Loss: 1.

Epoch: 0825, Total Run Time: 00:15:11 - Loss: 1.1795e+00, Accuracy: 0.629, Test Loss: 1.1217e+00, Test Accuracy: 0.614
Epoch: 0850, Total Run Time: 00:15:38 - Loss: 1.1785e+00, Accuracy: 0.627, Test Loss: 1.1277e+00, Test Accuracy: 0.609
Epoch: 0875, Total Run Time: 00:16:05 - Loss: 1.1774e+00, Accuracy: 0.630, Test Loss: 1.1190e+00, Test Accuracy: 0.614
Epoch: 0900, Total Run Time: 00:16:32 - Loss: 1.1792e+00, Accuracy: 0.629, Test Loss: 1.1019e+00, Test Accuracy: 0.618
Epoch: 0925, Total Run Time: 00:16:59 - Loss: 1.1765e+00, Accuracy: 0.632, Test Loss: 1.1171e+00, Test Accuracy: 0.613
Epoch: 0950, Total Run Time: 00:17:26 - Loss: 1.1773e+00, Accuracy: 0.630, Test Loss: 1.1100e+00, Test Accuracy: 0.618
Epoch: 0975, Total Run Time: 00:17:53 - Loss: 1.1784e+00, Accuracy: 0.630, Test Loss: 1.1220e+00, Test Accuracy: 0.610
Epoch: 1000, Total Run Time: 00:18:20 - Loss: 1.1773e+00, Accuracy: 0.630, Test Loss: 1.1068e+00, Test Accuracy: 0.618
Epoch: 1025, Total Run Time: 00:18:47 - Loss: 1.

Epoch: 1250, Total Run Time: 00:22:32 - Loss: 1.0373e+00, Accuracy: 0.677, Test Loss: 1.1100e+00, Test Accuracy: 0.624
Epoch: 1275, Total Run Time: 00:22:59 - Loss: 1.0368e+00, Accuracy: 0.679, Test Loss: 1.0980e+00, Test Accuracy: 0.632
FINISHED TRAINING: conv_net_depth_5_width_4
STARTING TRAINING: conv_net_depth_5_width_5
Epoch: 0000, Total Run Time: 00:00:02 - Loss: 1.8922e+00, Accuracy: 0.341, Test Loss: 1.7138e+00, Test Accuracy: 0.373
Epoch: 0025, Total Run Time: 00:00:29 - Loss: 1.1311e+00, Accuracy: 0.652, Test Loss: 1.0912e+00, Test Accuracy: 0.626
Epoch: 0050, Total Run Time: 00:00:56 - Loss: 1.0583e+00, Accuracy: 0.677, Test Loss: 1.0692e+00, Test Accuracy: 0.632
Epoch: 0075, Total Run Time: 00:01:23 - Loss: 1.0203e+00, Accuracy: 0.689, Test Loss: 1.0848e+00, Test Accuracy: 0.622
Epoch: 0100, Total Run Time: 00:01:50 - Loss: 9.9861e-01, Accuracy: 0.694, Test Loss: 1.0845e+00, Test Accuracy: 0.629
Epoch: 0125, Total Run Time: 00:02:17 - Loss: 9.8113e-01, Accuracy: 0.699, Test

Epoch: 0350, Total Run Time: 00:06:17 - Loss: 7.7202e-01, Accuracy: 0.759, Test Loss: 1.2125e+00, Test Accuracy: 0.628
Epoch: 0375, Total Run Time: 00:06:44 - Loss: 7.6786e-01, Accuracy: 0.759, Test Loss: 1.1862e+00, Test Accuracy: 0.636
Epoch: 0400, Total Run Time: 00:07:11 - Loss: 7.6548e-01, Accuracy: 0.760, Test Loss: 1.2169e+00, Test Accuracy: 0.623
Epoch: 0425, Total Run Time: 00:07:38 - Loss: 7.5956e-01, Accuracy: 0.760, Test Loss: 1.1982e+00, Test Accuracy: 0.631
Epoch: 0450, Total Run Time: 00:08:05 - Loss: 7.5708e-01, Accuracy: 0.763, Test Loss: 1.2037e+00, Test Accuracy: 0.629
Epoch: 0475, Total Run Time: 00:08:32 - Loss: 7.5466e-01, Accuracy: 0.764, Test Loss: 1.2227e+00, Test Accuracy: 0.626
Epoch: 0500, Total Run Time: 00:08:59 - Loss: 7.5630e-01, Accuracy: 0.761, Test Loss: 1.2129e+00, Test Accuracy: 0.630
Epoch: 0525, Total Run Time: 00:09:26 - Loss: 7.5149e-01, Accuracy: 0.764, Test Loss: 1.2236e+00, Test Accuracy: 0.625
Epoch: 0550, Total Run Time: 00:09:53 - Loss: 7.

...

Epoch: 0475, Total Run Time: 00:19:39 - Loss: 8.8290e-05, Accuracy: 1.000, Test Loss: 1.5175e+00, Test Accuracy: 0.727
Epoch: 0500, Total Run Time: 00:20:41 - Loss: 8.2247e-05, Accuracy: 1.000, Test Loss: 1.5226e+00, Test Accuracy: 0.726
Epoch: 0525, Total Run Time: 00:21:43 - Loss: 7.8347e-05, Accuracy: 1.000, Test Loss: 1.5277e+00, Test Accuracy: 0.728
Epoch: 0550, Total Run Time: 00:22:45 - Loss: 7.5483e-05, Accuracy: 1.000, Test Loss: 1.5337e+00, Test Accuracy: 0.726
Epoch: 0575, Total Run Time: 00:23:47 - Loss: 7.2343e-05, Accuracy: 1.000, Test Loss: 1.5409e+00, Test Accuracy: 0.726
Epoch: 0600, Total Run Time: 00:24:49 - Loss: 6.8016e-05, Accuracy: 1.000, Test Loss: 1.5456e+00, Test Accuracy: 0.726
Epoch: 0625, Total Run Time: 00:25:51 - Loss: 6.3526e-05, Accuracy: 1.000, Test Loss: 1.5528e+00, Test Accuracy: 0.728
Epoch: 0650, Total Run Time: 00:26:53 - Loss: 6.0526e-05, Accuracy: 1.000, Test Loss: 1.5551e+00, Test Accuracy: 0.728
Epoch: 0675, Total Run Time: 00:27:55 - Loss: 5.

Epoch: 0900, Total Run Time: 00:37:02 - Loss: 4.1360e-05, Accuracy: 1.000, Test Loss: 1.5220e+00, Test Accuracy: 0.728
Epoch: 0925, Total Run Time: 00:38:04 - Loss: 3.8719e-05, Accuracy: 1.000, Test Loss: 1.5240e+00, Test Accuracy: 0.729
Epoch: 0950, Total Run Time: 00:39:05 - Loss: 3.8536e-05, Accuracy: 1.000, Test Loss: 1.5275e+00, Test Accuracy: 0.729
Epoch: 0975, Total Run Time: 00:40:06 - Loss: 3.9255e-05, Accuracy: 1.000, Test Loss: 1.5312e+00, Test Accuracy: 0.728
Epoch: 1000, Total Run Time: 00:41:08 - Loss: 3.5785e-05, Accuracy: 1.000, Test Loss: 1.5345e+00, Test Accuracy: 0.729
Epoch: 1025, Total Run Time: 00:42:09 - Loss: 3.6068e-05, Accuracy: 1.000, Test Loss: 1.5372e+00, Test Accuracy: 0.728
Epoch: 1050, Total Run Time: 00:43:10 - Loss: 3.4876e-05, Accuracy: 1.000, Test Loss: 1.5392e+00, Test Accuracy: 0.728
Epoch: 1075, Total Run Time: 00:44:12 - Loss: 3.3508e-05, Accuracy: 1.000, Test Loss: 1.5408e+00, Test Accuracy: 0.729
Epoch: 1100, Total Run Time: 00:45:13 - Loss: 3.

Epoch: 0000, Total Run Time: 00:00:04 - Loss: 2.5628e+00, Accuracy: 0.301, Test Loss: 1.6598e+00, Test Accuracy: 0.418
Epoch: 0025, Total Run Time: 00:01:21 - Loss: 1.5707e-02, Accuracy: 1.000, Test Loss: 9.8330e-01, Test Accuracy: 0.732
Epoch: 0050, Total Run Time: 00:02:38 - Loss: 1.7186e-03, Accuracy: 1.000, Test Loss: 1.1172e+00, Test Accuracy: 0.740
Epoch: 0075, Total Run Time: 00:03:55 - Loss: 8.7640e-04, Accuracy: 1.000, Test Loss: 1.1811e+00, Test Accuracy: 0.738
Epoch: 0100, Total Run Time: 00:05:12 - Loss: 5.8155e-04, Accuracy: 1.000, Test Loss: 1.2225e+00, Test Accuracy: 0.737
Epoch: 0125, Total Run Time: 00:06:29 - Loss: 4.3400e-04, Accuracy: 1.000, Test Loss: 1.2538e+00, Test Accuracy: 0.737
Epoch: 0150, Total Run Time: 00:07:46 - Loss: 3.4310e-04, Accuracy: 1.000, Test Loss: 1.2761e+00, Test Accuracy: 0.736
Epoch: 0175, Total Run Time: 00:09:03 - Loss: 2.8143e-04, Accuracy: 1.000, Test Loss: 1.2962e+00, Test Accuracy: 0.735
Epoch: 0200, Total Run Time: 00:10:20 - Loss: 2.

Epoch: 0425, Total Run Time: 00:22:19 - Loss: 9.3019e-05, Accuracy: 1.000, Test Loss: 1.3833e+00, Test Accuracy: 0.737
Epoch: 0450, Total Run Time: 00:23:38 - Loss: 8.8150e-05, Accuracy: 1.000, Test Loss: 1.3922e+00, Test Accuracy: 0.736
Epoch: 0475, Total Run Time: 00:24:56 - Loss: 8.1478e-05, Accuracy: 1.000, Test Loss: 1.3951e+00, Test Accuracy: 0.736
Epoch: 0500, Total Run Time: 00:26:15 - Loss: 7.6846e-05, Accuracy: 1.000, Test Loss: 1.4041e+00, Test Accuracy: 0.736
Epoch: 0525, Total Run Time: 00:27:34 - Loss: 7.0853e-05, Accuracy: 1.000, Test Loss: 1.4097e+00, Test Accuracy: 0.736
Epoch: 0550, Total Run Time: 00:28:52 - Loss: 6.7473e-05, Accuracy: 1.000, Test Loss: 1.4150e+00, Test Accuracy: 0.736
Epoch: 0575, Total Run Time: 00:30:11 - Loss: 6.5358e-05, Accuracy: 1.000, Test Loss: 1.4201e+00, Test Accuracy: 0.735
Epoch: 0600, Total Run Time: 00:31:30 - Loss: 6.1094e-05, Accuracy: 1.000, Test Loss: 1.4248e+00, Test Accuracy: 0.736
Epoch: 0625, Total Run Time: 00:32:48 - Loss: 5.

Epoch: 0850, Total Run Time: 00:45:28 - Loss: 4.1220e-05, Accuracy: 1.000, Test Loss: 1.4049e+00, Test Accuracy: 0.745
Epoch: 0875, Total Run Time: 00:46:48 - Loss: 4.0746e-05, Accuracy: 1.000, Test Loss: 1.4085e+00, Test Accuracy: 0.744
Epoch: 0900, Total Run Time: 00:48:08 - Loss: 3.9800e-05, Accuracy: 1.000, Test Loss: 1.4119e+00, Test Accuracy: 0.744
Epoch: 0925, Total Run Time: 00:49:28 - Loss: 3.7051e-05, Accuracy: 1.000, Test Loss: 1.4136e+00, Test Accuracy: 0.744
Epoch: 0950, Total Run Time: 00:50:48 - Loss: 3.5117e-05, Accuracy: 1.000, Test Loss: 1.4158e+00, Test Accuracy: 0.744
Epoch: 0975, Total Run Time: 00:52:08 - Loss: 3.5636e-05, Accuracy: 1.000, Test Loss: 1.4198e+00, Test Accuracy: 0.744
Epoch: 1000, Total Run Time: 00:53:28 - Loss: 3.4437e-05, Accuracy: 1.000, Test Loss: 1.4224e+00, Test Accuracy: 0.744
Epoch: 1025, Total Run Time: 00:54:48 - Loss: 3.4100e-05, Accuracy: 1.000, Test Loss: 1.4237e+00, Test Accuracy: 0.744
Epoch: 1050, Total Run Time: 00:56:08 - Loss: 3.

## Graph Results: Train & Test Loss and Accuracy at both end of training and at optimal early stopping.

In [ ]:
# Plot the train/test loss and accuracy at the end of training for each model. 
widths = []
train_losses = []
train_accuracy = []
test_losses = []
test_accuracy = []


for model_id, history in depth_5_metrics.items():
    widths.append(int(model_id[23:]))
    
    train_losses.append(hstry.get('loss'))
    train_accuracy.append(hstry.get('accuracy'))
    test_losses.append(hstry.get('val_loss'))
    test_accuracy.append(hstry.get('val_accuracy'))    
    
train_losses = np.array(train_losses)
train_accuracy = np.array(train_accuracy)
test_losses = np.array(test_losses)
test_accuracy = np.array(test_accuracy)

# optimal early stopping values
optimal_test_idx = test_accuracy.argmax(axis=1)
optimal_early_train_losses = np.array([train_losses[i, idx] for i, idx in enumerate(optimal_test_idx)])
optimal_early_train_accuracy = np.array([train_accuracy[i, idx] for i, idx in enumerate(optimal_test_idx)])
optimal_early_test_losses = np.array([test_losses[i, idx] for i, idx in enumerate(optimal_test_idx)])
optimal_early_test_accuracy = np.array([test_accuracy[i, idx] for i, idx in enumerate(optimal_test_idx)])

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))

train_loss_plt = axes[0][0]
test_loss_plt = axes[0][1]
train_accy_plt = axes[1][0]
test_accy_plt = axes[1][1]

fig.suptitle('Depth 3 Conv Net Results:', fontsize=24, fontweight='bold')

train_loss_plt.plot(widths, train_losses[:,-1], marker='o', markersize=4, label='Final Train Loss')
train_loss_plt.plot(widths, optimal_early_train_losses, marker='o', markersize=4, label='Optimal Early Stopping Train Loss')
train_loss_plt.set_ylabel('Train Loss', fontsize=16)

test_loss_plt.plot(widths, test_losses[:,-1], marker='o', markersize=4, label='Final Test Loss')
test_loss_plt.plot(widths, optimal_early_test_losses, marker='o', markersize=4, label='Optimal Early Stopping Test Loss')
test_loss_plt.set_ylabel('Test Loss', fontsize=16)

train_accy_plt.plot(widths, 100*(1 - train_accuracy[:,-1]), marker='o', markersize=4, label='Final Train Accuracy')
train_accy_plt.plot(widths, 100*(1 - optimal_early_train_accuracy), marker='o', markersize=4, label='Optimal Early Stopping Train Accuracy')
train_accy_plt.set_ylabel('Train Accuracy', fontsize=16)

test_accy_plt.plot(widths, 100*(1 - test_accuracy[:,-1]), marker='o', markersize=4, label='Final Test Accuracy')
test_accy_plt.plot(widths, 100*(1 - optimal_early_test_accuracy), marker='o', markersize=4, label='Optimal Early Stopping Test Accuracy')
test_accy_plt.set_ylabel('Test Accuracy', fontsize=16)



x_ticks = [1, 16, 32, 64, 96, 128, 192, 256]

for ax in axes.flatten():
    ax.set_xlabel('Layer Width', fontsize=16)
    ax.set_xticks(x_ticks)
    ax.legend(fontsize=14)
    ax.grid(alpha=0.5)

fig.tight_layout(pad=1.15, h_pad=2)
plt.show()